# Explora y consulta un DataFrame con Spark SQL

## Contexto
Una de las habilidades más importantes para trabajar con Spark SQL es la **exploración de DataFrames** y la realización de **consultas eficientes**, ya sea con métodos de la API o con sentencias SQL.

## Consigna
A partir de un archivo CSV o JSON provisto por el docente (por ejemplo, datos de empleados o ventas), **carga los datos en un DataFrame**, explora su estructura y realiza al menos **tres consultas relevantes** utilizando Spark SQL.

## Paso a Paso
1. Lee el archivo CSV o JSON con `spark.read.format().load()`.
2. Muestra las primeras filas con `.show()` y analiza su estructura con `.printSchema()` y `.describe()`.
3. Registra el DataFrame como vista temporal con `.createOrReplaceTempView()`.
4. Ejecuta consultas SQL con `spark.sql()`, por ejemplo:
   - Filtrar por condiciones (ej: ventas > 1000)
   - Agrupar y calcular promedios
   - Ordenar por columnas específicas
5. Muestra y comenta los resultados.



In [23]:
from pyspark.sql import SparkSession

# Crear SparkSession
spark = SparkSession.builder \
    .appName("ExploracionDataFrame") \
    .getOrCreate()


In [24]:
from google.colab import files
uploaded = files.upload()  # Te pedirá seleccionar el CSV desde tu PC


Saving 1. datos_agricolas.csv to 1. datos_agricolas (1).csv


In [25]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("sep", ";") \
    .option("encoding", "ISO-8859-1") \
    .load("1. datos_agricolas.csv")


In [26]:
# Renombrar columnas para Spark SQL
df = df.withColumnRenamed("Año", "Ano") \
       .withColumnRenamed("Región", "Region") \
       .withColumnRenamed("Producción_Ton", "Produccion_Ton")


In [27]:
print(df.columns)
df.show(5)


['Ano', 'Mes', 'Region', 'Cultivo', 'Superficie_Ha', 'Produccion_Ton', 'Precio_Ton', 'Costos_Insumos', 'Ingresos', 'Utilidad']
+----+---+------+-------+-------------+--------------+----------+--------------+--------+--------+
| Ano|Mes|Region|Cultivo|Superficie_Ha|Produccion_Ton|Precio_Ton|Costos_Insumos|Ingresos|Utilidad|
+----+---+------+-------+-------------+--------------+----------+--------------+--------+--------+
|2023|  7|   Sur|Manzana|           34|          4583|       378|          5267| 1730770| 1725503|
|2024|  5|Centro| Tomate|          253|          3501|       106|          6988|  372681|  365693|
|2023|  3|   Sur|   Soja|          430|           716|       249|          9680|  178327|  168647|
|2023|  3|   Sur|   Soja|           37|          3074|       241|          2485|  739850|  737365|
|2023|  4| Norte|   Maíz|          462|           763|       317|         15713|  242169|  226456|
+----+---+------+-------+-------------+--------------+----------+--------------+-

In [28]:
# Mostrar la estructura del DataFrame
df.printSchema()


root
 |-- Ano: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Cultivo: string (nullable = true)
 |-- Superficie_Ha: integer (nullable = true)
 |-- Produccion_Ton: integer (nullable = true)
 |-- Precio_Ton: integer (nullable = true)
 |-- Costos_Insumos: integer (nullable = true)
 |-- Ingresos: integer (nullable = true)
 |-- Utilidad: integer (nullable = true)



In [29]:
df.createOrReplaceTempView("agricolas")


In [30]:
consulta1 = spark.sql("""
    SELECT Ano, Mes, Region, Cultivo, Produccion_Ton, Ingresos, Utilidad
    FROM agricolas
    WHERE Produccion_Ton > 4000
""")
consulta1.show(5)


+----+---+------+-------+--------------+--------+--------+
| Ano|Mes|Region|Cultivo|Produccion_Ton|Ingresos|Utilidad|
+----+---+------+-------+--------------+--------+--------+
|2023|  7|   Sur|Manzana|          4583| 1730770| 1725503|
|2023| 11|   Sur|Manzana|          4030| 1597291| 1578128|
|2023|  2|   Sur|  Trigo|          4312| 1044410| 1033103|
|2023|  5|Centro|   Papa|          4438|  981286|  962784|
|2024|  2|   Sur|    Uva|          4167| 1325231| 1306189|
+----+---+------+-------+--------------+--------+--------+
only showing top 5 rows



In [31]:
consulta2 = spark.sql("""
    SELECT Region, AVG(Ingresos) AS Promedio_Ingresos
    FROM agricolas
    GROUP BY Region
""")
consulta2.show()


+------+-----------------+
|Region|Promedio_Ingresos|
+------+-----------------+
|   Sur|713160.0016611295|
| Norte|773226.0250783699|
|Centro|701692.3839285715|
+------+-----------------+



In [32]:
consulta3 = spark.sql("""
    SELECT Cultivo, SUM(Utilidad) AS Utilidad_Total
    FROM agricolas
    GROUP BY Cultivo
    ORDER BY Utilidad_Total DESC
    LIMIT 5
""")
consulta3.show()


+-------+--------------+
|Cultivo|Utilidad_Total|
+-------+--------------+
|Manzana|     206003764|
| Tomate|     195542018|
|    Uva|     193713685|
|   Papa|     190117853|
|   Soja|     182046485|
+-------+--------------+

